# Extract data from Google Calendar

### Description:
Assume that you work and get paid based on hours you have worked. You note working hours in your Google Calendar, and you send your working ours to your manager in every xy time period. This program creates a file with the working hours you can send directly saving time on doing it "by hand".

### Usage:
To use this program you have to create a Google CalendarAPI. (See links below.) For the first use you have to sign in to your Google account, but after that you can save the credentials you need to log in so you don't need to sign in every time.

In your calendar consistent naming is essential. You have to name all events the same.

As some work late at night the last days work can be finished at the following days morning (like 1am or 2am) the code counts these to the previous day (when work started).

At the end the program counts the total time of work in hours.

Output: hours.txt

Credits: [Google Developers](https://developers.google.com/calendar/quickstart/python), [Towards Data Science](https://towardsdatascience.com/accessing-google-calendar-events-data-using-python-e915599d3ae2)

In [1]:
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle
import datetime
import calendar

In [ ]:
# For first time use you will need to log in to your Google account
# then you can save credentials for loging in for next times
scopes = ['https://www.googleapis.com/auth/calendar.readonly']

flow = InstalledAppFlow.from_client_secrets_file("client_secret.json", scopes=scopes)
credentials = flow.run_console()

In [2]:
# Saving / loading credentials -> avoid loging in every time
#pickle.dump(credentials, open("token.pkl", "wb")) 
credentials = pickle.load(open("token.pkl", "rb"))

In [3]:
service = build("calendar", "v3", credentials=credentials)

In [4]:
result = service.calendarList().list().execute()

In [5]:
calendar_id = result['items'][1]['id']

In [6]:
events = service.events().list(calendarId=calendar_id).execute()

In [9]:
def yourHoursPlease():
    def strToDatetime(string):
        # type of date-time stored in calendar: '2019-10-14T16:00:00+01:00'
        try:
            result = datetime.datetime.strptime(string, '%Y-%m-%dT%H:%M:%S+01:00')
        except:
            result = datetime.datetime.strptime(string, '%Y-%m-%dT%H:%M:%SZ')
        return result
    
    nameOfWork = input('How do you call work events in your calendar? ')
    
    startMonth = input('Starting date\'s month: ')
    startDay = input('Startind date\'s day: ')

    endMonth = input('Ending date\'s month: ')
    endDay = input('Ending date\'s day: ')

    start = datetime.datetime(2019, int(startMonth), int(startDay),0,0,0).isoformat() + 'Z'
    end = datetime.datetime(2019, int(endMonth), int(endDay) + 1, 2,59,59).isoformat() + 'Z' # last day's work can be ended on,
                                                                   # following day in the morning (1-2 am time)
        
    # Getting all events from calendar in the above set time period
    events_result = service.events().list(calendarId='primary', timeMin=start,
                                            timeMax=end, singleEvents=True,
                                            orderBy='startTime').execute()
    
    # Delete all redundant events:
    events = events_result['items'] # list of the events
    for event in events:
        if event['summary'] != nameOfWork:
            events.remove(event)
    # --------------------------------------------
    lastEventsWeek = 0
    lastEventsDay = 0
    total = datetime.timedelta(0) # total hours
    
    firstRow = True # avoid line break in the first row
    
    file = open('hours.txt', "w")
    for event in events:
        start = strToDatetime(event['start']['dateTime'])
        end = strToDatetime(event['end']['dateTime'])
        total = total + (end - start)
        if start.weekday() < lastEventsWeek:
            file.write('\n--------------------')
        
        lastEventsWeek = start.weekday()

        if strToDatetime(event['start']['dateTime']).day != lastEventsDay:
            if firstRow:
                firstRow = False
            else:
                file.write('\n')
            file.write(str(start.day) + '. ')
            file.write(calendar.day_name[start.weekday()]+': ')
        else:
            file.write(' + ')
        if start.minute == 0:
            startMinute = '00'
        else:
            startMinute = str(start.minute)
        if end.minute == 0:
            endMinute = '00'
        else:
            endMinute = str(end.minute)

        file.write(str(start.hour) + ':' + startMinute + '-')
        file.write(str(end.hour) + ':' + endMinute)
        lastEventsDay = start.day
    
    result = total.days * 24 + total.seconds / 3600
    file.write(f'\n________________________ {result} hours in total.')
    print(f'{result} hours in total.')
    file.close()
    return result

In [10]:
yourHoursPlease()

How do you call work events in your calendar? Munka
Starting date's month: 10
Startind date's day: 14
Ending date's month: 10
Ending date's day: 27
85.5 hours in total.


85.5